### Goal & Problem Statement
Si Dudung punya akun Twitter dan Instagram dengan followers diatas 50K. Dengan konten utamanya dia yaitu nge-share2 info diskon apapun termasuk diskon2 dari LinkAja. Gimana caranya dia dapetin info diskon dari LinkAja secara efektif?

### Approaches
- Tiap hari di jam tertentu dia harus buka aplikasi LinkAja buat liat diskon apa aja yg baru rilis.
  - Pros: gampang, tinggal buka aplikasi
  - Cons: repetitif, bayangin kalo dia punya 20+ platform/konten yg harus di cek satu2 setiap hari. Sama sekali ga realtime
- Nungguin notifikasi dari aplikasi yg ngasih tau info diskon.
  - Pros: udah dikasih tau, tinggal nunggu notif yg masuk tanpa perlu buka apps nya buat ngecek manual. Bisa jadi info diskon benar2 realtime dan fresh.
  - Cons: ga semua info di blast. Notif bisa ketumpuk sama notif lain, dan terkadang main asal swipe buat clear notif
- Buat automasi supaya dia bisa dapetin info diskon.
  - Pros: otomatis, bisa dikirim ke Telegram atau disimpen di database. Gaperlu ngelakuin hal repetitif juga.
  - Cons: perlu develop si automasi tsb. perlu cost tambahan buat sewa server. ga realtime2 amat.

In [ ]:
import os
import time
import json

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

## Chrome Options

In [ ]:
chrome_options = webdriver.ChromeOptions()  
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36")
chrome_options.add_argument("window-size=1024,720")
# chrome_options.add_argument("--headless")

In [ ]:
driver = webdriver.Chrome(executable_path=os.path.abspath("chromedriver"), options=chrome_options)

In [ ]:
URL = "https://www.linkaja.id/merchants/linkaja"

## Let's scrape!

In [ ]:
driver.get(URL)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
articles = soup.find_all(attrs={"class": "card card-promo animated fadeInUp delayp1"}) # by class
result = []

for idx, article in enumerate(articles):
    link = article["href"]
    driver.execute_script('window.open("{}","_blank");'.format(link)) # open new tab
    driver.switch_to.window(driver.window_handles[1])
    WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".container")))

    html_detail_page = driver.page_source
    detail_page = BeautifulSoup(html_detail_page, 'html.parser')

    title = detail_page.select_one(".container").select_one("h1").text # builder pattern
    content = detail_page.select_one("div.content > div > div.col-md-7.animated.fadeInUp.delayp5 > p:nth-child(2) > span > span").text # querySelector

    driver.close()
    driver.switch_to.window(driver.window_handles[0])

    print("[TITLE] {}".format(title))
    print("[CONTENT]\n{}\n".format(content))
    print("-----------------\n")
    
    result.append({
        "title": title,
        "content": content
    })
    
    # do something else
    
    if idx == 2:
        break
        
with open('data.json', 'w') as f:
    json.dump(result, f)
    
driver.close()